In [52]:
import pandas as pd
print(pd.__version__)

2.2.2


In [ ]:
df = pd.read_json('data/pcr_udeska.json')

def extract_nazev_all(informace):
    if isinstance(informace, list):
        return [item.get('název', {}).get('cs') for item in informace if 'název' in item]
    return None

dfNazev= df['informace'].apply(extract_nazev_all).dropna()
dfe = dfNazev

dfe.to_json("pcr_iformace.json", force_ascii=False)
print(dfe.head(10))